In [1]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import numpy as np
import pandas as pd
from keras.models import load_model
BATCH_SIZE = 16
NUM_CLASSES = 10
NUM_EPOCHS = 10


Using TensorFlow backend.


In [2]:
class Game:

    def __init__(self, size=4, score_to_win=None, rate_2=0.5, random=False, enable_rewrite_board=False):
        '''

        :param size: the size of the board
        :param score_to_win: the terminate score to indicate `win`
        :param rate_2: the probability of the next element to be 2 (otherwise 4)
        :param random: a random initialized board (a harder mode)
        '''
        self.size = size
        if score_to_win is None:
            score_to_win = np.inf
        self.score_to_win = score_to_win
        self.__rate_2 = rate_2
        if random:
            self.__board = \
                2 ** np.random.randint(1, 10, size=(self.size, self.size))
            self.__end = False
        else:
            self.__board = np.zeros((self.size, self.size))
            # initilize the board (with 2 entries)
            self._maybe_new_entry()
            self._maybe_new_entry()
        self.enable_rewrite_board = enable_rewrite_board
        assert not self.end

    def move(self, direction):
        '''
        direction:
            0: left
            1: down
            2: right
            3: up
        '''
        # treat all direction as left (by rotation)
        board_to_left = np.rot90(self.board, -direction)
        for row in range(self.size):
            core = _merge(board_to_left[row])
            board_to_left[row, :len(core)] = core
            board_to_left[row, len(core):] = 0

        # rotation to the original
        self.__board = np.rot90(board_to_left, direction)
        self._maybe_new_entry()

    def __str__(self):
        board = "State:"
        for row in self.board:
            board += ('\t' + '{:8d}' *
                      self.size + '\n').format(*map(int, row))
        board += "Score: {0:d}".format(self.score)
        return board

    @property
    def board(self):
        '''`NOTE`: Setting board by indexing,
        i.e. board[1,3]=2, will not raise error.'''
        return self.__board.copy()

    @board.setter
    def board(self, x):
        if self.enable_rewrite_board:
            assert self.__board.shape == x.shape
            self.__board = x.astype(self.__board.dtype)
        else:
            print("Disable to rewrite `board` manually.")

    @property
    def score(self):
        return int(self.board.max())

    @property
    def end(self):
        '''
        0: continue
        1: lose
        2: win
        '''
        if self.score >= self.score_to_win:
            return 2
        elif self.__end:
            return 1
        else:
            return 0

    def _maybe_new_entry(self):
        '''maybe set a new entry 2 / 4 according to `rate_2`'''
        where_empty = self._where_empty()
        if where_empty:
            selected = where_empty[np.random.randint(0, len(where_empty))]
            self.__board[selected] = \
                2 if np.random.random() < self.__rate_2 else 4
            self.__end = False
        else:
            self.__end = True

    def _where_empty(self):
        '''return where is empty in the board'''
        return list(zip(*np.where(self.board == 0)))


def _merge(row):
    '''merge the row, there may be some improvement'''
    non_zero = row[row != 0]  # remove zeros
    core = [None]
    for elem in non_zero:
        if core[-1] is None:
            core[-1] = elem
        elif core[-1] == elem:
            core[-1] = 2 * elem
            core.append(None)
        else:
            core.append(elem)
    if core[-1] is None:
        core.pop()
    return core


In [3]:
out_shape = (4,4)
CAND = 16
map_table = {2**i:i for i in range(1,CAND)}
map_table[0]=0

def NB_board(arr):
    ret = np.zeros(shape=out_shape+(CAND,),dtype=bool)
    for r in range (out_shape[0]):
        for c in range (out_shape[1]):
            ret[r,c,map_table[arr[r,c]]]=1
    ret=ret.reshape(1,4,4,16)
    return ret

In [4]:
from IPython.display import HTML, display as ipy_display
import pandas as pd
class Display:
    '''A basic display.'''

    def display(self, game):
        if game.end == 2:
            self.win(game)
        elif game.end == 1:
            self.lose(game)
        else:
            self.show(game)


    def _display(self, game):
        print(game)
       # print(game.board)
       # dataframe = pd.DataFrame({'board':game.board.reshape(1,)},index=[0])
       # dataframe.to_csv("train.csv",index = False,sep=',')
    def show(self, game):
        self._display(game)

    def win(self, game):
        self._display(game)
        print("You win! Score: %s" % game.score)

    def lose(self, game):
        self._display(game)
        print("You lose! Score: %s" % game.score)


class IPythonDisplay(Display):
    '''A better display for IPython (Jupyter) notebook environments.'''

    def __init__(self, display_size=40):
        self.display_size = display_size

    def _render(self, game):
        board = game.board
        html = '''<h1>Score: {}</h1>'''.format(game.score)
        table = '''<table style="border: 5px solid black;">{}</table>'''
        td = '''<td style="border:3px solid black; text-align:center;"
         width="%s" height="%s">{}</td>''' % (self.display_size, self.display_size)
        content = ''
        for row in range(game.size):
            content += '''<tr>'''
            for col in range(game.size):
                elem = int(board[row, col])
                content += td.format(elem if elem else "")
            content += '''</tr>'''
        html += table.format(content)
        return html

    def _display(self, game):
        if 'ipykernel' in sys.modules:
            source = self._render(game)
            ipy_display(HTML(source))
        else:
            print("Warning: since it's not in ipykernel, "
                  "it will show the command line version.")
            super()._display(game)


In [5]:

class the_model_512:
    def __init__(self):
        self.model=model_512
    def predict(self,board):
        Real_board = NB_board(board)
        direction = self.model.predict(Real_board)
        move = direction.argmax()
        return move

In [6]:

import csv
class Agent:
    '''Agent Base.'''
    def __init__(self, game, display=None):
        self.game = game
        self.display = display

    def play(self, max_iter=np.inf, verbose=False):
        n_iter = 0
        while (n_iter < max_iter) and (not self.game.end):
            direction = self.step()
            self.game.move(direction)
            n_iter += 1
            if verbose:
                #print("Iter: {}".format(n_iter))
                #print("======Direction: {}======".format(
                    #["left", "down", "right", "up"][direction]))
                if self.display is not None:
                    self.display.display(self.game)
        #print("Iter: {}".format(n_iter))
        
    def step(self):
        direction = int(input("0: left, 1: down, 2: right, 3: up = ")) % 4
        return direction

In [ ]:
class Best_Agent(Agent):
    def __init__(self, game, display=None): 
        if game.size != 4:
            raise ValueError(
                "`%s` can only work with game of `size` 4." % self.__class__.__name__)
        super().__init__(game, display)

        self.search_func_512 = the_model_512()
        from expectimax import board_to_move
        self.search_func1 = board_to_move
          
    def step(self):
        
        direction = self.search_func_512.predict(self.game.board) 
        direction1 = self.search_func1(self.game.board)
        l=[0]*16
        k=0
        for i in range(0,4):
            for j in range(0,4): 
                l[k]=self.game.board[i][j]
                k=k+1
        l.append(direction1)
        with open ('evolution_finally.csv',"a") as csvfile:
            writer =csv.writer(csvfile)
            writer.writerow(l)
            csvfile.close()
        
        return direction
    

In [ ]:

model_512 = load_model('learn_finally.h5')
times_128 = 0
times_256 = 0
times_512 = 0
times_1024 = 0
#move = 0
for j in range (0,25000):
    
    game = Game(4, random=False)
    agent = Best_Agent(game)
    agent.play(verbose=True)
    print (game.score) 
    
    if game.score >=128:
        times_128 = times_128+1
    if game.score >=256:
        times_256 = times_256 + 1
    if game.score >=512:
        times_512 = times_512 + 1
    if game.score >=1024:
        times_1024 = times_1024 + 1
    data_new = pd.read_csv('evolution_finally.csv',header=None)
    x=data_new.iloc[:,0:16]
    y=data_new.iloc[:,16]

    x=np.array(x)
    x=x.reshape(x.shape[0],4,4)
    
    x_train=np.zeros((x.shape[0],4,4,16))
    y_train=np.zeros((y.shape[0],4))
    
    for i in range (0,x.shape[0]):
        x_train[i]=NB_board(x[i])
        c=np.zeros((1,4))
        c[0][y[i]]=1
        y_train[i]=c[0]
        
    print(x_train.shape)
            
    loss,acc = model_512.train_on_batch(x_train, y_train)
    print('loss=',float(loss),'acc=',float(acc))
    model_512.save('learn_finally.h5')
    with open ('evolution_finally.csv',"w") as csvfile:
        csvfile.close()
    #if game.score >=1024:
        #times_1024 = times_1024 + 1
        
    print("到达128分次数为：",times_128)
    print("到达256分次数为：",times_256)
    print("到达512分次数为：",times_512)
    print("到达1024分次数为：",times_1024)
    print("----------------------------------第",j+1,"轮训练———————————————————")

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Loaded expectmax lib for 2048: /home/u20976/Untitled Folder/game2048/expectimax/bin/2048.so
64
(135, 4, 4, 16)
loss= 0.8575815558433533 acc= 0.5555555820465088
到达128分次数为： 0
到达256分次数为： 0
到达512分次数为： 0
到达1024分次数为： 0
----------------------------------第 1 轮训练———————————————————
1024
(397, 4, 4, 16)
loss= 0.8507519960403442 acc= 0.6120907068252563
到达128分次数为： 1
到达256分次数为： 1
到达512分次数为： 1
到达1024分次数为： 1
----------------------------------第 2 轮训练———————————————————
1024
(691, 4, 4, 16)
loss= 0.7850354313850403 acc= 0.59913170337677
到达128分次数为： 2
到达256分次数为： 2
到达512分次数为： 2
到达1024分次数为： 2
----------------------------------第 3 轮训练———————————————————
512
(453, 4, 4, 16)
loss= 0.8247770071029663 acc= 0.6225165724754333
到达128分次数为： 3
到达256分次数为： 3
到达512分次数为： 3
到达1024分次数为： 2
-------------------

1024
(398, 4, 4, 16)
loss= 0.7736988067626953 acc= 0.6331658363342285
到达128分次数为： 38
到达256分次数为： 35
到达512分次数为： 32
到达1024分次数为： 18
----------------------------------第 39 轮训练———————————————————
512
(291, 4, 4, 16)
loss= 0.8122459650039673 acc= 0.5635738968849182
到达128分次数为： 39
到达256分次数为： 36
到达512分次数为： 33
到达1024分次数为： 18
----------------------------------第 40 轮训练———————————————————
1024
(542, 4, 4, 16)
loss= 0.8177989721298218 acc= 0.5922509431838989
到达128分次数为： 40
到达256分次数为： 37
到达512分次数为： 34
到达1024分次数为： 19
----------------------------------第 41 轮训练———————————————————
1024
(748, 4, 4, 16)
loss= 0.8516656160354614 acc= 0.5855615139007568
到达128分次数为： 41
到达256分次数为： 38
到达512分次数为： 35
到达1024分次数为： 20
----------------------------------第 42 轮训练———————————————————
512
(403, 4, 4, 16)
loss= 0.756847620010376 acc= 0.6401985287666321
到达128分次数为： 42
到达256分次数为： 39
到达512分次数为： 36
到达1024分次数为： 20
----------------------------------第 43 轮训练———————————————————
256
(210, 4, 4, 16)
loss= 0.7778221964836121 acc= 0.666666

1024
(549, 4, 4, 16)
loss= 0.8205010890960693 acc= 0.6010928750038147
到达128分次数为： 82
到达256分次数为： 77
到达512分次数为： 67
到达1024分次数为： 32
----------------------------------第 83 轮训练———————————————————
1024
(440, 4, 4, 16)
loss= 0.8050981163978577 acc= 0.6272727251052856
到达128分次数为： 83
到达256分次数为： 78
到达512分次数为： 68
到达1024分次数为： 33
----------------------------------第 84 轮训练———————————————————
512
(211, 4, 4, 16)
loss= 0.7960625886917114 acc= 0.6255924105644226
到达128分次数为： 84
到达256分次数为： 79
到达512分次数为： 69
到达1024分次数为： 33
----------------------------------第 85 轮训练———————————————————
1024
(484, 4, 4, 16)
loss= 1.0484024286270142 acc= 0.5061983466148376
到达128分次数为： 85
到达256分次数为： 80
到达512分次数为： 70
到达1024分次数为： 34
----------------------------------第 86 轮训练———————————————————
512
(342, 4, 4, 16)
loss= 0.9004611968994141 acc= 0.6023392081260681
到达128分次数为： 86
到达256分次数为： 81
到达512分次数为： 71
到达1024分次数为： 34
----------------------------------第 87 轮训练———————————————————
256
(168, 4, 4, 16)
loss= 0.7443718910217285 acc= 0.64285

到达128分次数为： 124
到达256分次数为： 116
到达512分次数为： 100
到达1024分次数为： 52
----------------------------------第 126 轮训练———————————————————
1024
(420, 4, 4, 16)
loss= 0.7562064528465271 acc= 0.6404761672019958
到达128分次数为： 125
到达256分次数为： 117
到达512分次数为： 101
到达1024分次数为： 53
----------------------------------第 127 轮训练———————————————————
512
(383, 4, 4, 16)
loss= 0.8049682378768921 acc= 0.5979112386703491
到达128分次数为： 126
到达256分次数为： 118
到达512分次数为： 102
到达1024分次数为： 53
----------------------------------第 128 轮训练———————————————————
256
(138, 4, 4, 16)
loss= 0.767491340637207 acc= 0.6304348111152649
到达128分次数为： 127
到达256分次数为： 119
到达512分次数为： 102
到达1024分次数为： 53
----------------------------------第 129 轮训练———————————————————
512
(315, 4, 4, 16)
loss= 0.7817103266716003 acc= 0.6698412895202637
到达128分次数为： 128
到达256分次数为： 120
到达512分次数为： 103
到达1024分次数为： 53
----------------------------------第 130 轮训练———————————————————
512
(298, 4, 4, 16)
loss= 0.7839500904083252 acc= 0.6208053827285767
到达128分次数为： 129
到达256分次数为： 121
到达512分次数为：

到达128分次数为： 167
到达256分次数为： 157
到达512分次数为： 137
到达1024分次数为： 67
----------------------------------第 169 轮训练———————————————————
1024
(569, 4, 4, 16)
loss= 0.7466909289360046 acc= 0.6590509414672852
到达128分次数为： 168
到达256分次数为： 158
到达512分次数为： 138
到达1024分次数为： 68
----------------------------------第 170 轮训练———————————————————
512
(342, 4, 4, 16)
loss= 0.8003459572792053 acc= 0.5789473652839661
到达128分次数为： 169
到达256分次数为： 159
到达512分次数为： 139
到达1024分次数为： 68
----------------------------------第 171 轮训练———————————————————
1024
(496, 4, 4, 16)
loss= 0.8423821330070496 acc= 0.6088709831237793
到达128分次数为： 170
到达256分次数为： 160
到达512分次数为： 140
到达1024分次数为： 69
----------------------------------第 172 轮训练———————————————————
1024
(677, 4, 4, 16)
loss= 0.8016210198402405 acc= 0.6174298524856567
到达128分次数为： 171
到达256分次数为： 161
到达512分次数为： 141
到达1024分次数为： 70
----------------------------------第 173 轮训练———————————————————
1024
(622, 4, 4, 16)
loss= 0.8435898423194885 acc= 0.6141479015350342
到达128分次数为： 172
到达256分次数为： 162
到达512分

...

1024
(498, 4, 4, 16)
loss= 0.8603802919387817 acc= 0.5582329034805298
到达128分次数为： 338
到达256分次数为： 327
到达512分次数为： 274
到达1024分次数为： 137
----------------------------------第 341 轮训练———————————————————
512
(409, 4, 4, 16)
loss= 0.8129029870033264 acc= 0.6283618807792664
到达128分次数为： 339
到达256分次数为： 328
到达512分次数为： 275
到达1024分次数为： 137
----------------------------------第 342 轮训练———————————————————
128
(87, 4, 4, 16)
loss= 0.7527624368667603 acc= 0.6551724076271057
到达128分次数为： 340
到达256分次数为： 328
到达512分次数为： 275
到达1024分次数为： 137
----------------------------------第 343 轮训练———————————————————
1024
(661, 4, 4, 16)
loss= 0.8384891748428345 acc= 0.5854765772819519
到达128分次数为： 341
到达256分次数为： 329
到达512分次数为： 276
到达1024分次数为： 138
----------------------------------第 344 轮训练———————————————————
2048
(918, 4, 4, 16)
loss= 0.8474504351615906 acc= 0.6045751571655273
到达128分次数为： 342
到达256分次数为： 330
到达512分次数为： 277
到达1024分次数为： 139
----------------------------------第 345 轮训练———————————————————
1024
(474, 4, 4, 16)
loss= 0.8387

1024
(437, 4, 4, 16)
loss= 0.7340555787086487 acc= 0.6292906403541565
到达128分次数为： 381
到达256分次数为： 369
到达512分次数为： 313
到达1024分次数为： 158
----------------------------------第 384 轮训练———————————————————
512
(354, 4, 4, 16)
loss= 0.8220207095146179 acc= 0.6384180784225464
到达128分次数为： 382
到达256分次数为： 370
到达512分次数为： 314
到达1024分次数为： 158
----------------------------------第 385 轮训练———————————————————
512
(293, 4, 4, 16)
loss= 0.7619454264640808 acc= 0.6313993334770203
到达128分次数为： 383
到达256分次数为： 371
到达512分次数为： 315
到达1024分次数为： 158
----------------------------------第 386 轮训练———————————————————
1024
(480, 4, 4, 16)
loss= 0.8831409215927124 acc= 0.574999988079071
到达128分次数为： 384
到达256分次数为： 372
到达512分次数为： 316
到达1024分次数为： 159
----------------------------------第 387 轮训练———————————————————
1024
(695, 4, 4, 16)
loss= 0.7459484338760376 acc= 0.6575539708137512
到达128分次数为： 385
到达256分次数为： 373
到达512分次数为： 317
到达1024分次数为： 160
----------------------------------第 388 轮训练———————————————————
512
(308, 4, 4, 16)
loss= 0.72588

512
(346, 4, 4, 16)
loss= 0.7586947679519653 acc= 0.615606963634491
到达128分次数为： 423
到达256分次数为： 411
到达512分次数为： 347
到达1024分次数为： 173
----------------------------------第 427 轮训练———————————————————
1024
(458, 4, 4, 16)
loss= 0.846944272518158 acc= 0.5938864350318909
到达128分次数为： 424
到达256分次数为： 412
到达512分次数为： 348
到达1024分次数为： 174
----------------------------------第 428 轮训练———————————————————
2048
(739, 4, 4, 16)
loss= 0.8774040937423706 acc= 0.5778078436851501
到达128分次数为： 425
到达256分次数为： 413
到达512分次数为： 349
到达1024分次数为： 175
----------------------------------第 429 轮训练———————————————————
512
(340, 4, 4, 16)
loss= 0.8274985551834106 acc= 0.6176470518112183
到达128分次数为： 426
到达256分次数为： 414
到达512分次数为： 350
到达1024分次数为： 175
----------------------------------第 430 轮训练———————————————————
512
(329, 4, 4, 16)
loss= 0.7483361959457397 acc= 0.6565349698066711
到达128分次数为： 427
到达256分次数为： 415
到达512分次数为： 351
到达1024分次数为： 175
----------------------------------第 431 轮训练———————————————————
512
(321, 4, 4, 16)
loss= 0.8470289

128
(175, 4, 4, 16)
loss= 1.0080941915512085 acc= 0.49714285135269165
到达128分次数为： 465
到达256分次数为： 452
到达512分次数为： 383
到达1024分次数为： 189
----------------------------------第 470 轮训练———————————————————
512
(379, 4, 4, 16)
loss= 0.7276131510734558 acc= 0.6358839273452759
到达128分次数为： 466
到达256分次数为： 453
到达512分次数为： 384
到达1024分次数为： 189
----------------------------------第 471 轮训练———————————————————
512
(403, 4, 4, 16)
loss= 0.8034483194351196 acc= 0.6377171277999878
到达128分次数为： 467
到达256分次数为： 454
到达512分次数为： 385
到达1024分次数为： 189
----------------------------------第 472 轮训练———————————————————
512
(342, 4, 4, 16)
loss= 0.8134856820106506 acc= 0.6345029473304749
到达128分次数为： 468
到达256分次数为： 455
到达512分次数为： 386
到达1024分次数为： 189
----------------------------------第 473 轮训练———————————————————
512
(263, 4, 4, 16)
loss= 0.7757878303527832 acc= 0.6501901149749756
到达128分次数为： 469
到达256分次数为： 456
到达512分次数为： 387
到达1024分次数为： 189
----------------------------------第 474 轮训练———————————————————
256
(198, 4, 4, 16)
loss= 0.969884

1024
(474, 4, 4, 16)
loss= 0.794920802116394 acc= 0.6012658476829529
到达128分次数为： 507
到达256分次数为： 494
到达512分次数为： 421
到达1024分次数为： 211
----------------------------------第 513 轮训练———————————————————
2048
(860, 4, 4, 16)
loss= 0.8449358344078064 acc= 0.5930232405662537
到达128分次数为： 508
到达256分次数为： 495
到达512分次数为： 422
到达1024分次数为： 212
----------------------------------第 514 轮训练———————————————————
1024
(472, 4, 4, 16)
loss= 0.8406994342803955 acc= 0.5953390002250671
到达128分次数为： 509
到达256分次数为： 496
到达512分次数为： 423
到达1024分次数为： 213
----------------------------------第 515 轮训练———————————————————
